# Implementation of ResNet18

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

print(torch.__version__)

2.1.2


## device related functions

In [34]:
def num_gpus():
    """Get the number of GPUs."""
    if torch.cuda.is_available():
        return torch.cuda.device_count()
    return 0


def try_gpu(i=0):
    if num_gpus() > i:
        return torch.device(f"cuda:{i}")
    else:
        print(f"gpu with index '{i}' is not available")
        return torch.device("cpu")


def try_all_gpus():
    """
    Return all available GPUs, or [cpu(),] if no GPU exists.

    Returns:
    -------
    devices: list
        A list of devices.
    """

    return [torch.device(f"cuda:{i}") for i in range(num_gpus())]


def gpus_info(gpu_list):
    for i, gpu in enumerate(gpu_list):
        print(f"""gpu {i}: {torch.cuda.get_device_name(gpu)}""")


def current_gpu_info():
    current_device = torch.cuda.current_device()
    print(f"""current gpu : {torch.cuda.get_device_name(current_device)}""")


try_gpu(2)
gpu_list = try_all_gpus()
gpus_info(gpu_list)

torch.cuda.set_device(0)
current_gpu_info()

tensor_A = torch.tensor([1,2,3]).to(try_gpu(0))
device = tensor_A.device
print(device)


gpu with index '2' is not available
gpu 0: NVIDIA GeForce GTX 1650
current gpu : NVIDIA GeForce GTX 1650
cuda:0


## ResNet

### the Residual Block

![The ResNet-18 architecture.](..\ResNet18\images\materials\resnet-block.svg)

In [40]:
class ResidualBlock(nn.Module):
    def __init__(self, num_channels, use_1x1conv=False, strides=1):
        super().__init__()
        self.convolution_layer_1 = nn.LazyConv2d(
            num_channels, kernel_size=3, padding=(1, 1), stride=strides
        )
        self.convolution_layer_2 = nn.LazyConv2d(num_channels, kernel_size=3, padding=(1, 1))
        if use_1x1conv:
            self.convolution_layer_3 = nn.LazyConv2d(num_channels, kernel_size=1, stride=strides)
        else:
            self.convolution_layer_3 = None
        self.batch_norm_layer_1 = nn.LazyBatchNorm2d()
        self.batch_norm_layer_2 = nn.LazyBatchNorm2d()

    def forward(self,X):
        Y = F.relu(self.batch_norm_layer_1(self.convolution_layer_1(X)))
        Y = self.batch_norm_layer_2(self.convolution_layer_2(Y))
        if self.convolution_layer_3:
            X = self.convolution_layer_3(X)
        Y += X
        return F.relu(Y)     

In [43]:
residual_block_1 = ResidualBlock(3) # a residual block with 3 output channels
block_input = torch.rand(4,3,6,6) # 4 samples each with 3 channels whose width*height is 6*6
residual_block_1(block_input).shape

torch.Size([4, 3, 6, 6])

In [45]:
# use the 1x1conv to adjust the original data
residual_block_2 = ResidualBlock(10, use_1x1conv=True) # a residual block with 10 output channels
block_input = torch.rand(4,3,6,6) # 4 samples each with 3 channels whose width*height is 6*6
residual_block_2(block_input).shape

f:\anaconda\envs\FYP\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([4, 10, 6, 6])

### the whole model

![The ResNet-18 architecture.](../ResNet18\images\materials\resnet18-90.svg)

In [ ]:
class ResNet(nn.Module):
    
